In [1]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
data = pd.read_csv('data.csv') #Three stocks (R,X_s,X_b) Without predictors

In [4]:
#Initialize Neural Network and set-up the placeholders
tf.reset_default_graph()
NN_input = tf.placeholder(shape=[1,2],dtype=tf.float32)
NN_weights = tf.Variable(tf.random_uniform([2,10],0,0.01))
Q_FA = tf.matmul(NN_input,NN_weights)
A_Max = tf.argmax(Q_FA,1)

# Calculate loss for the NN from the Q values
Q_Next = tf.placeholder(shape=[1,10],dtype=tf.float32)
loss = tf.reduce_sum(tf.square(Q_Next - Q_FA))
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(loss)

#Define Action Matrix (Now discrete case) 
A = np.linspace(0,1,10) # portfolio weights of stocks (1-weight) is the weight in the bonds

In [5]:
# initialize
init = tf.global_variables_initializer()
gamma = 1 
epsilon = 0.1
train_data = 241 # also an expanding window
Index = []
OptimalWeights = []
# data parsing
dates = data['Date']
mdata = data[['r','xs','xb']]
mdata.index = pd.DatetimeIndex(dates)
n = data.size/4-4
periods = 60
epochs = 50

with tf.Session() as sess:
    sess.run(init) # only initialize the Neural Network once

    # Train the network
    currentEpoch = 0
    while currentEpoch < epochs:
        a_old = 0
        for i in range(0,train_data):
            s = mdata.iloc[i,1:4].values.reshape(1,2)
            #Choose an action by greedily (with e chance of random action) from the Q-network
            a_int,allQ = sess.run([A_Max,Q_FA],feed_dict={NN_input:s})
            a = A[a_int-1]  # -1 because the output neurons are labeled 1 till 101 and it will be an index
            if np.random.rand(1) < epsilon:
                a = random.choice(A)

            #Get new state and reward from environment
            s1 = data.iloc[i+1,2:4].values.reshape(1,2)
            r = (a*s1[0][0] + (1-a)*s1[0][1] - 0.1*abs(a_old-a)) #reward: this is now the wealth gained from this step, but could be other rewards like utility
            Q = sess.run(Q_FA,feed_dict={NN_input:s1})
            a_old = a
            #Obtain maxQ' and set our target value for chosen action.
            Q1 = np.max(Q)
            targetQ = allQ
            targetQ[0,a_int] = r + gamma*Q1

            #Train the neural network using target and predicted Q values
            _,W1 = sess.run([updateModel,NN_input],feed_dict={NN_input:s,Q_Next:targetQ})

            if currentEpoch  > 100:
                # decrease amount of random actions over time in order to improve exploitation rather than exploration
                # only increase exploitation when a good action has been found (otherwise one exploits a bad solution)
                e = 1./((i/50) + 10)
                break
        currentEpoch += 1
        
    # Calculate the optimal weights
    for i in range(train_data,int(n)):
        print(i)
        
        s = mdata.iloc[i,1:4].values.reshape(1,2)
        a_int,allQ = sess.run([A_Max,Q_FA],feed_dict={NN_input:s})
        aOpt = A[a_int-1]
        OptimalWeights.append(aOpt[0])
        Index.append(i)
        
        a = aOpt[0]
        # Now Update the Network with the latest observation
        s1 = mdata.iloc[i+1,1:4].values.reshape(1,2)
        r = (a*s1[0][0] + (1-a)*s1[0][1]- 0.1*abs(a_old-a)) #reward: this is now the wealth gained from this step, but could be other rewards like utility
        a_old = a
        Q = sess.run(Q_FA,feed_dict={NN_input:s1})
        Q1 = np.max(Q)
        targetQ = allQ
        targetQ[0,a_int] = r + gamma*Q1
        _,W1 = sess.run([updateModel,NN_input],feed_dict={NN_input:s,Q_Next:targetQ})
        
print('Writing away results')
df = pd.DataFrame({'index date':Index,'Optimal Weights':OptimalWeights, 'xs':mdata.iloc[241:int(n)]['xs'],'xb':mdata.iloc[241:int(n)]['xb']})
df.to_excel('Results_NN(General)_g10_TC_e50.xlsx', sheet_name='sheet1', index=False)

    

241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
